### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### QUERIES

#### QUERY 30 DIAS

In [3]:
# 17GB
q_30 = '''SELECT 

orders.city_id,
orders.city_name,
orders.total_orders,
orders.confirmed_orders,
orders.confirmed_orders_logistics,
orders.rejected_by_restaurant,
orders.rejected_by_peya,
orders.cancelled_by_user,
orders.users,
orders.acquisitions,
orders.activity_rate,
orders.restaurant_orders,
orders.vertical_orders,
orders.campaign_orders,
orders.confirmed_orders_voucher,
orders.vendor_late_numerator,
orders.vendor_late_denominator,
orders.avg_vendor_late,
orders.avg_delivery_time,
orders.dropoff_distance,
orders.pickup_distance,
orders.orders_less_20min,
orders.orders_late,
orders.stacked_orders,
orders.gfv_inc_peya_discount_logistics,
orders.take_in_commission_logistics,
orders.commission_logistics,
orders.take_in_marketplace,
orders.commission_marketplace,
orders.shipping_amount,
orders.avg_basket_size,
orders.rejected_by_rider,
supply.online_partners,
supply.high_performance_partners,
supply.low_performance_partners,
supply.zombies,
supply.kam,
supply.avg_orders,
supply.median_orders,
orders.kam_orders,
orders.concept_orders,
orders.delivery_free_orders,
marketing_investment.cambio_usd,
marketing_investment.marketing_investment,
marketing_investment.marketing_investment_usd,
marketing_investment.subsidized_orders,
marketing_investment.investment_subsidies,
marketing_investment.cpo_campaigns,
sessions.sessions,
sessions.sessions_shops,
sessions.sessions_w_transaction,
sessions.cvr3,
sessions.cvr,
sessions.direct_sessions,
sessions.crm_sessions,
sessions.cpc_sessions,
sessions.organic_sessions

FROM 

        (SELECT 

            orders.address.area.city. id AS city_id,
            cities.city_name,

            -- órdenes generales

            count(orders.order_id) AS total_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS confirmed_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND with_logistics = TRUE THEN orders.order_id ELSE NULL END) AS confirmed_orders_logistics,
            count(CASE WHEN fail_rate_owner = 'Restaurant' THEN orders.order_id ELSE NULL END) AS rejected_by_restaurant,
            count(CASE WHEN fail_rate_owner = 'PedidosYa' THEN orders.order_id ELSE NULL END) AS rejected_by_peya,
            count(CASE WHEN fail_rate_owner = 'User' THEN orders.order_id ELSE NULL END) AS cancelled_by_user,
            count(CASE WHEN fail_rate_owner = 'Rider' THEN orders.order_id ELSE NULL END) AS rejected_by_rider, 

            -- demand y usuarios

            count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.user.id ELSE NULL END) AS users,
            count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' AND data.first = TRUE THEN orders.user.id ELSE NULL END) AS acquisitions,
            SAFE_DIVIDE(CAST(count(CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric),
            CAST(count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.user.id ELSE NULL END) AS numeric)) AS activity_rate,


            -- verticales,vouchers,campañas
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND orders.business_type_id = 1 THEN orders.order_id ELSE NULL END) AS restaurant_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND orders.business_type_id != 1 THEN orders.order_id ELSE NULL END) AS vertical_orders,
            count(CASE WHEN subsidized_order IS NOT NULL THEN orders.order_id ELSE NULL END) AS campaign_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND has_voucher_discount = 1 THEN orders.order_id ELSE NULL END) AS confirmed_orders_voucher,



            -- logística   

            count(CASE WHEN logs.is_vendor_late_10 = 1 THEN orders.order_id ELSE NULL END) AS vendor_late_numerator,
            count(CASE WHEN logs.is_vendor_late_nn = 1 THEN orders.order_id ELSE NULL END) AS vendor_late_denominator,
            avg(CASE WHEN logs.is_vendor_late_nn = 1 THEN SAFE_CAST(logs.timings.vendor_late AS numeric)/60 ELSE NULL END) AS avg_vendor_late,
            avg(CASE WHEN logs.timings.actual_delivery_time IS NOT NULL THEN SAFE_CAST(logs.timings.actual_delivery_time AS numeric)/60 ELSE NULL END) AS avg_delivery_time,
            avg(CASE WHEN orders.order_status = 'CONFIRMED' AND dropoff_distance_manhattan IS NOT NULL THEN dropoff_distance_manhattan ELSE NULL END) AS dropoff_distance,
            avg(CASE WHEN orders.order_status = 'CONFIRMED' AND pickup_distance_manhattan IS NOT NULL THEN pickup_distance_manhattan ELSE NULL END) AS pickup_distance,
            count(CASE WHEN SAFE_CAST(logs.timings.actual_delivery_time AS numeric)/60 <= 20 AND logs.timings.actual_delivery_time IS NOT NULL AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS orders_less_20min,
            count(CASE WHEN logs.is_order_late_10 = 1 AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS orders_late,
            count(CASE WHEN logistic_deliveries.stacked_deliveries > 0 AND logistic_deliveries.stacked_deliveries IS NOT NULL AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS stacked_orders,


            -- economics

            sum(CASE WHEN with_logistics = TRUE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                        / CAST(count(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS gfv_inc_peya_discount_logistics,
                        
            sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) 
                        / CAST(count(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS take_in_commission_logistics,
                        
            sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) /
            sum(CASE WHEN with_logistics = TRUE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                        AS commission_logistics,
                        
            sum(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) 
                        / CAST(count(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS take_in_marketplace,
                        
            sum(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) /
            sum(CASE WHEN with_logistics = FALSE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                        AS commission_marketplace,
                        
            avg(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.shipping_amount ELSE NULL END) AS shipping_amount,
                        
            avg(CASE WHEN orders.order_status = 'CONFIRMED' THEN total_amount+orders.shipping_amount ELSE NULL END) AS avg_basket_size,
                        
            sum(CASE WHEN orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END)+sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.shipping_amount ELSE NULL END) AS total_revenue,
            
            -- extras

            count(CASE WHEN franchise.franchise_name	 
                            
                            IN ("McDonald's",'Grido','Mostaza','Burger King','El Club de la Milanesa',
                            "Lucciano's",'Fabric Sushi','Faricci','Kiosco de Empanadas','Cremolatti','Morita','Freddo','Kentucky',
                            'Sándwich del Club','Subway','KFC','El Noble','Tomasso','Solo Empanadas','Betos Lomos','El Hornito Santiagueño',
                            'Heladería Los Amores','Temple Bar','Hell´s Pizza','Sushi Pop','Almacén de Pizzas',"Wendy's",
                            'Fábrica de Pizzas','Lomitos 2x1','SushiClub','Holy Mila','Tea Connection','Green Eat','Che Sushi',
                            'Che Muzza','Vía Bana Helados','Helados Arlequín','Sei Tu','Flymorfi','Kentucky Café','Inside Tea Connection',
                            'Bien de Campo','La Causa Nikkei','Va Pizza','Questa Pizza','Citadeli','Rollstar','El Viejo Zoilo Argentina',
                            'Guapaletas Argentina','Mercado Milanesa','Bunka Sushi','Sabor Gourmet','Noble Gourmet','El Gran Cheff',
                            'Poke Pop','Tigre Morado','Green Eat Market','Terra Di Pasta','Pizza Break','Dandy','Grangys Congelados',
                            'Green Eat Sushi & Poke','Mamá Cora','Il Salvatore') AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS kam_orders,
                            
                        count(CASE WHEN is_concept = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS concept_orders,
                        
                        count(CASE WHEN orders.order_status = 'CONFIRMED' AND with_logistics = TRUE AND coalesce(orders.shipping_amount,0) = 0 THEN orders.order_id ELSE NULL END) AS delivery_free_orders


            FROM `peya-bi-tools-pro.il_core.fact_orders` orders

                LEFT JOIN `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS logs
                
                    ON orders.order_id = logs.peya_order_id
                        
                LEFT JOIN (
                        
                            SELECT peya_order_id, sum(stacked_deliveries) AS stacked_deliveries, avg(pickup_distance_manhattan) AS pickup_distance_manhattan,avg(dropoff_distance_manhattan) AS dropoff_distance_manhattan
                            
                            FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` logs

                            LEFT JOIN UNNEST(logs.deliveries) AS details
                            
                            GROUP BY peya_order_id
                            
                            ) logistic_deliveries

                    ON orders.order_id = logistic_deliveries.peya_order_id

                
                
                -- CAMPAIGN ORDERS

                    LEFT JOIN (SELECT DISTINCT order_id AS subsidized_order

                                    FROM `peya-bi-tools-pro.il_core.fact_orders` o

                                    LEFT JOIN UNNEST(o.details) AS details


                                    JOIN peya-bi-tools-pro.il_core.dim_subsidized_product AS dim_subProd     
                                    ON details.product.product_id = dim_subProd.product_id

                                    WHERE o.registered_date >= DATE_SUB(current_date(), INTERVAL 31 day)
                                    AND o.country_id = 3) campaigns
                    
                    ON orders.order_id = campaigns.subsidized_order
            
                
                -- PARTNERS

                    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` partners

                    ON orders.restaurant.id = partners.partner_id


                -- CITIES

                LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` cities

                ON orders.address.area.city.id = cities.city_id



            WHERE 

            orders.registered_date >= DATE_SUB(current_date(), INTERVAL 31 day)
            AND orders.country_id = 3

            GROUP BY 
            1,2

        ) orders


-- 2 supply

LEFT JOIN (

            
        SELECT
        
        
        city_id,
        count(partner_id) AS online_partners,
        count(CASE WHEN confirmed_orders >= 200 THEN partner_id ELSE NULL END) AS high_performance_partners,
        count(CASE WHEN confirmed_orders <= 30 THEN partner_id ELSE NULL END) AS low_performance_partners,
        count(CASE WHEN confirmed_orders = 0 THEN partner_id ELSE NULL END) AS zombies,
        count(CASE WHEN kam = 'KAM' THEN partner_id ELSE NULL END) AS kam,
        avg(confirmed_orders) AS avg_orders,
        median_orders
        
        
        FROM
        
        
                (
                
                SELECT 
                
                partners.partner_id,
                city_id,
                coalesce(confirmed_orders,0) AS confirmed_orders,
                percentile_cont(coalesce(confirmed_orders,0),0.5) OVER (PARTITION BY city_id) AS median_orders,
                
                if(franchise.franchise_name	 
                
                IN ("McDonald's",'Grido','Mostaza','Burger King','El Club de la Milanesa',
                "Lucciano's",'Fabric Sushi','Faricci','Kiosco de Empanadas','Cremolatti','Morita','Freddo','Kentucky',
                'Sándwich del Club','Subway','KFC','El Noble','Tomasso','Solo Empanadas','Betos Lomos','El Hornito Santiagueño',
                'Heladería Los Amores','Temple Bar','Hell´s Pizza','Sushi Pop','Almacén de Pizzas',"Wendy's",
                'Fábrica de Pizzas','Lomitos 2x1','SushiClub','Holy Mila','Tea Connection','Green Eat','Che Sushi',
                'Che Muzza','Vía Bana Helados','Helados Arlequín','Sei Tu','Flymorfi','Kentucky Café','Inside Tea Connection',
                'Bien de Campo','La Causa Nikkei','Va Pizza','Questa Pizza','Citadeli','Rollstar','El Viejo Zoilo Argentina',
                'Guapaletas Argentina','Mercado Milanesa','Bunka Sushi','Sabor Gourmet','Noble Gourmet','El Gran Cheff',
                'Poke Pop','Tigre Morado','Green Eat Market','Terra Di Pasta','Pizza Break','Dandy','Grangys Congelados',
                'Green Eat Sushi & Poke','Mamá Cora','Il Salvatore')
                
                ,'KAM','Normal') AS kam
                
                
                
                FROM `peya-bi-tools-pro.il_core.dim_partner` partners
                
                /* órdenes por partner*/
                            
                    LEFT JOIN (SELECT
                                restaurant.id AS partner_id,
                                count(order_id) AS confirmed_orders
                            
                                FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                                        
                                        
                                WHERE
                                registered_date >= DATE_SUB(current_date(), INTERVAL 31 day)
                                AND orders.country_id = 3
                                AND order_status = 'CONFIRMED'
                                        
                                GROUP BY 
                                1
                                
                                ) orders
                                
                                ON partners.partner_id = orders.partner_id
                                
                                
                WHERE 
                
                country_id = 3
                AND is_online = TRUE
                
                )
        
        
        GROUP BY city_id, median_orders

        ORDER BY 1
        
        
    ) supply

ON orders.city_id = supply.city_id

-- 3 MARKETING INVESTMENT

LEFT JOIN (

        SELECT 
    
            address.area.city. id AS city_id,
            usd AS cambio_usd,
            
            sum(CASE WHEN discounts.discount_paid_by = 'COMPANY' AND discount_type_name IN ('VOUCHER','SUBSIDIZED') THEN discounts. discount_amount ELSE NULL END) AS marketing_investment, 
            SAFE_DIVIDE(sum(CASE WHEN discounts.discount_paid_by = 'COMPANY' AND discount_type_name IN ('VOUCHER','SUBSIDIZED') THEN discounts. discount_amount ELSE NULL END),usd) AS marketing_investment_usd, 
            count(CASE WHEN discounts.discount_paid_by = 'COMPANY' AND discount_type_name IN ('VOUCHER','SUBSIDIZED') THEN discounts. order_id	 ELSE NULL END) AS subsidized_orders,
            SAFE_DIVIDE(sum(CASE WHEN discount_type_name = 'SUBSIDIZED' AND discounts.discount_paid_by = 'COMPANY' THEN discounts. discount_amount ELSE NULL END),usd) AS investment_subsidies,
            
            SAFE_DIVIDE(SAFE_DIVIDE(sum(CASE WHEN discount_type_name = 'SUBSIDIZED'AND  discounts.discount_paid_by = 'COMPANY' THEN discounts. discount_amount ELSE NULL END),usd),
            COUNT(CASE WHEN discount_type_name = 'SUBSIDIZED' AND discounts.discount_paid_by = 'COMPANY' THEN discounts. order_id	 ELSE NULL END)) AS cpo_campaigns,
            
            SAFE_DIVIDE(sum(CASE WHEN discounts.discount_paid_by = 'RESTAURANT' THEN discounts. discount_amount ELSE NULL END),usd) AS investment_by_others_usd,
            COUNT(CASE WHEN discounts.discount_paid_by = 'RESTAURANT' THEN discounts. order_id	 ELSE NULL END) AS subsidized_orders_by_others
            
            
            FROM `peya-bi-tools-pro.il_core.fact_orders` o

            LEFT JOIN UNNEST (o.discounts) discounts

            -- para sacar el valor del dolaruco
            LEFT JOIN (SELECT currency_exchange_date, rate_us AS usd FROM `peya-bi-tools-pro.il_core.dim_currency_exchange`

            WHERE currency_id = 3) exchange
                                            
            ON date_trunc(registered_date, month) = exchange.currency_exchange_date


            WHERE
            registered_date >= DATE_SUB(current_date(), INTERVAL 31 day)
            AND country_id = 3

            GROUP BY 1,2

        ) marketing_investment

ON orders.city_id = marketing_investment.city_id

-- 4 SESSIONS

LEFT JOIN (

            SELECT 
        
        cdcity,
        count(visitid) AS sessions,
        
        -- cvrs
        
        count(CASE WHEN uniqueshop >= 1 THEN visitid ELSE NULL END) AS sessions_shops,
        count(CASE WHEN transaction >= 1 THEN visitid ELSE NULL END) AS sessions_w_transaction,
        SAFE_DIVIDE(count(CASE WHEN transaction >= 1 THEN visitid ELSE NULL END),CAST(count(CASE WHEN uniqueshop >= 1 THEN visitid ELSE NULL END) AS numeric)) AS cvr3,
        SAFE_DIVIDE(count(CASE WHEN transaction >= 1 THEN visitid ELSE NULL END),CAST(count(visitid) AS numeric)) AS cvr,
        
        -- session source
        count(CASE WHEN source = '(direct)' THEN visitid ELSE NULL END) AS direct_sessions,
        count(CASE WHEN lower(medium) = 'crm' THEN visitid ELSE NULL END) AS crm_sessions,
        count(CASE WHEN medium = 'cpc' THEN visitid ELSE NULL END) AS cpc_sessions,
        count(CASE WHEN medium = 'organic' THEN visitid ELSE NULL END) AS organic_sessions,
        
        -- cvrs hour
        
        SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=0 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<11  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=0 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<11  THEN visitid ELSE null END) AS numeric)) AS cvr_manana,
        SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=11 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<15  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=11 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<15  THEN visitid ELSE null END) AS numeric)) AS cvr_mediodia,
        SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=15 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<19  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=15 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<19  THEN visitid ELSE null END) AS numeric)) AS cvr_tarde,
        SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=19 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<24  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=19 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<24  THEN visitid ELSE null END) AS numeric)) AS cvr_noche
        
        
        FROM `peya-bi-tools-pro.il_sessions.fact_sessions`
        
        
        WHERE 
        
        date_parsed >= DATE_SUB(current_date(), INTERVAL 31 day)
        AND cdCountry = 'Argentina'
        
        GROUP BY
        
        1
    ) sessions

ON orders.city_name = sessions.cdcity


ORDER BY 1'''

#### QUERY WEEKLY

In [4]:
# 27GB
q_weekly = '''SELECT 

orders.city_id,
orders.city_name,
orders.registered_week,
orders.total_orders,
orders.confirmed_orders,
orders.confirmed_orders_logistics,
orders.rejected_by_restaurant,
orders.rejected_by_peya,
orders.cancelled_by_user,
orders.users,
orders.acquisitions,
orders.activity_rate,
orders.restaurant_orders,
orders.vertical_orders,
orders.campaign_orders,
orders.confirmed_orders_voucher,
orders.vendor_late_numerator,
orders.vendor_late_denominator,
orders.avg_vendor_late,
orders.avg_delivery_time,
orders.dropoff_distance,
orders.pickup_distance,
orders.orders_less_20min,
orders.orders_late,
orders.stacked_orders,
nps.nps,
orders.gfv_inc_peya_discount_logistics,
orders.take_in_commission_logistics,
orders.commission_logistics,
orders.take_in_marketplace,
orders.commission_marketplace,
orders.shipping_amount,
orders.avg_basket_size,
supply.online_partners,
supply.od_partners,
supply.online_verticals,
supply.churned,
supply.new_online,
supply.zombies,
supply.high_orders_partners,
supply.cvr_partners,
nps.promoters,
nps.detractors,
nps.answers,
supply.new_online_logistic,
supply.new_online_vertical,
orders.rejected_by_rider

FROM 

-- 1 ORDERS

        (SELECT 

        address.area.city. id AS city_id,
        cities.city_name,
        DATE_TRUNC(registered_date, isoweek) AS registered_week,

        -- órdenes generales

        count(orders.order_id) AS total_orders,
        count(CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS confirmed_orders,
        count(CASE WHEN orders.order_status = 'CONFIRMED' AND with_logistics = TRUE THEN orders.order_id ELSE NULL END) AS confirmed_orders_logistics,
        count(CASE WHEN fail_rate_owner = 'Restaurant' THEN orders.order_id ELSE NULL END) AS rejected_by_restaurant,
        count(CASE WHEN fail_rate_owner = 'PedidosYa' THEN orders.order_id ELSE NULL END) AS rejected_by_peya,
        count(CASE WHEN fail_rate_owner = 'User' THEN orders.order_id ELSE NULL END) AS cancelled_by_user,
        count(CASE WHEN fail_rate_owner = 'Rider' THEN orders.order_id ELSE NULL END) AS rejected_by_rider, 

        -- demand y usuarios

        count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.user.id ELSE NULL END) AS users,
        count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' AND data.first = TRUE THEN orders.user.id ELSE NULL END) AS acquisitions,
        SAFE_DIVIDE(CAST(count(CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric),
        CAST(count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.user.id ELSE NULL END) AS numeric)) AS activity_rate,


        -- verticales,vouchers,campañas
        count(CASE WHEN orders.order_status = 'CONFIRMED' AND orders.business_type_id = 1 THEN orders.order_id ELSE NULL END) AS restaurant_orders,
        count(CASE WHEN orders.order_status = 'CONFIRMED' AND orders.business_type_id != 1 THEN orders.order_id ELSE NULL END) AS vertical_orders,
        count(CASE WHEN subsidized_order IS NOT NULL THEN orders.order_id ELSE NULL END) AS campaign_orders,
        count(CASE WHEN orders.order_status = 'CONFIRMED' AND has_voucher_discount = 1 THEN orders.order_id ELSE NULL END) AS confirmed_orders_voucher,



        -- logística   

        count(CASE WHEN logs.is_vendor_late_10 = 1 THEN orders.order_id ELSE NULL END) AS vendor_late_numerator,
        count(CASE WHEN logs.is_vendor_late_nn = 1 THEN orders.order_id ELSE NULL END) AS vendor_late_denominator,
        avg(CASE WHEN logs.is_vendor_late_nn = 1 THEN SAFE_CAST(logs.timings.vendor_late AS numeric)/60 ELSE NULL END) AS avg_vendor_late,
        avg(CASE WHEN logs.timings.actual_delivery_time IS NOT NULL THEN SAFE_CAST(logs.timings.actual_delivery_time AS numeric)/60 ELSE NULL END) AS avg_delivery_time,
        avg(CASE WHEN orders.order_status = 'CONFIRMED' AND dropoff_distance_manhattan IS NOT NULL THEN dropoff_distance_manhattan ELSE NULL END) AS dropoff_distance,
        avg(CASE WHEN orders.order_status = 'CONFIRMED' AND pickup_distance_manhattan IS NOT NULL THEN pickup_distance_manhattan ELSE NULL END) AS pickup_distance,
        count(CASE WHEN SAFE_CAST(logs.timings.actual_delivery_time AS numeric)/60 <= 20 AND logs.timings.actual_delivery_time IS NOT NULL AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS orders_less_20min,
        count(CASE WHEN logs.is_order_late_10 = 1 AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS orders_late,
        count(CASE WHEN logistic_deliveries.stacked_deliveries > 0 AND logistic_deliveries.stacked_deliveries IS NOT NULL AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS stacked_orders,


        -- economics

        sum(CASE WHEN with_logistics = TRUE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                    / CAST(count(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS gfv_inc_peya_discount_logistics,
                    
        sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) 
                    / CAST(count(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS take_in_commission_logistics,
                    
        sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) /
        sum(CASE WHEN with_logistics = TRUE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                    AS commission_logistics,
                    
        sum(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) 
                    / CAST(count(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS take_in_marketplace,
                    
        sum(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) /
        sum(CASE WHEN with_logistics = FALSE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                    AS commission_marketplace,
                    
        avg(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN shipping_amount ELSE NULL END) AS shipping_amount,
                    
        avg(CASE WHEN orders.order_status = 'CONFIRMED' THEN total_amount+shipping_amount ELSE NULL END) AS avg_basket_size,
                    
        sum(CASE WHEN orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END)+sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN shipping_amount ELSE NULL END) AS total_revenue
        



        FROM `peya-bi-tools-pro.il_core.fact_orders` orders

            LEFT JOIN `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS logs
            
                ON orders.order_id = logs.peya_order_id
                    
            LEFT JOIN (
                    
                        SELECT peya_order_id, sum(stacked_deliveries) AS stacked_deliveries, avg(pickup_distance_manhattan) AS pickup_distance_manhattan,avg(dropoff_distance_manhattan) AS dropoff_distance_manhattan
                        
                        FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` logs

                        LEFT JOIN UNNEST(logs.deliveries) AS details
                        
                        GROUP BY peya_order_id
                        
                        ) logistic_deliveries

                ON orders.order_id = logistic_deliveries.peya_order_id

            
            
            -- CAMPAIGN ORDERS

                LEFT JOIN (SELECT DISTINCT order_id AS subsidized_order

                                FROM `peya-bi-tools-pro.il_core.fact_orders` o

                                LEFT JOIN UNNEST(o.details) AS details


                                JOIN peya-bi-tools-pro.il_core.dim_subsidized_product AS dim_subProd     
                                ON details.product.product_id = dim_subProd.product_id

                                WHERE o.registered_date >= DATE_SUB(DATE_TRUNC(current_date(), isoweek), INTERVAL 10 week)
                                AND o.country_id = 3) campaigns
                
                ON orders.order_id = campaigns.subsidized_order

                -- CITIES

                LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` cities

                ON orders.address.area.city.id = cities.city_id
        WHERE 

        registered_date >= DATE_SUB(DATE_TRUNC(current_date(), isoweek), INTERVAL 10 week)
        AND cities.country_id = 3

        GROUP BY 
        1,2,3

    ) orders

-- 2 NPS

LEFT JOIN (

            SELECT
            
            address.area.city. id AS city_id,
            DATE_TRUNC(registered_date, isoweek) AS registered_week,
            
            -- nps
            avg(if(nps_score>=9,100,if(nps_score>=7,0,if(nps_score>=0 AND nps_score IS NOT NULL,-100,NULL)))) AS nps,
            count(CASE WHEN nps_score >= 9 AND nps_score IS NOT NULL THEN orders.order_id ELSE NULL END) AS promoters,
            count(CASE WHEN nps_score <= 6 AND nps_score IS NOT NULL THEN orders.order_id ELSE NULL END) AS detractors,
            count(CASE WHEN nps_score IS NOT NULL THEN orders.order_id ELSE NULL END) AS answers
            
            FROM `peya-bi-tools-pro.il_core.fact_orders` orders
            
            -- nps
            
            LEFT JOIN (SELECT DISTINCT order_id, nps_score FROM `peya-bi-tools-pro.il_nps.fact_nps_ao_related_kpis`) nps_ao_related_kpis
            
            ON orders.order_id = nps_ao_related_kpis.order_id
            
            WHERE 
            
            registered_date >= DATE_SUB(DATE_TRUNC(current_date(), isoweek), INTERVAL 10 week)
            AND country_id = 3
            
            
            GROUP BY
            
            1,2

            ORDER BY 1,2
    ) nps

    ON orders.city_id = nps.city_id AND orders.registered_week = nps.registered_week

-- 3 SUPPLY

LEFT JOIN (

        -- saca los datos de status por día para ver si estuvo online y algunas cosillas mas
                    
            WITH historical AS (

                SELECT 
                    
                    partners_historical.city_id,
                    DATE_TRUNC(DATE(full_date),isoweek) AS registered_week,
                    DATE(partners_historical.full_date) AS date,
                    partners_historical.restaurant_id,
                    partners_historical.is_online,
                    is_logistic,
                    is_new_online,
                    business_name,
                    sessions,
                    row_number() OVER (PARTITION BY DATE_TRUNC(DATE(full_date),isoweek),partners_historical.restaurant_id ORDER BY DATE(partners_historical.full_date) DESC) AS row_num
                    
                    FROM `peya-bi-tools-pro.il_core.dim_historical_partners` partners_historical
                    
                    
                    -- sesiones
                    
                    LEFT JOIN `peya-bi-tools-pro.il_sessions.fact_sessions_by_restaurant` sessions_by_restaurant
                    
                    ON partners_historical.restaurant_id = sessions_by_restaurant.restaurant_id
                    AND DATE(partners_historical.full_date) = sessions_by_restaurant.date
                    
                    WHERE 
                        DATE(partners_historical.full_date) >= DATE_SUB(DATE_TRUNC(current_date(), isoweek), INTERVAL 10 week)
                        AND partners_historical.country_id = 3

            )   


    SELECT
    
        city_id,
        registered_week,
        count(CASE WHEN was_online = TRUE THEN restaurant_id ELSE NULL END) AS online_partners,
        count(CASE WHEN was_online = TRUE AND is_logistic = TRUE THEN restaurant_id ELSE NULL END) AS od_partners,
        count(CASE WHEN was_online = TRUE AND business_name != 'Restaurant' THEN restaurant_id ELSE NULL END) AS online_verticals,
        count(CASE WHEN was_online = FALSE AND online_last_week = TRUE THEN restaurant_id ELSE NULL END) AS churned,
        count(CASE WHEN new_online = TRUE THEN restaurant_id ELSE NULL END) AS new_online,
        count(CASE WHEN new_online = TRUE AND is_logistic = TRUE THEN restaurant_id ELSE NULL END) AS new_online_logistic,
        count(CASE WHEN new_online = TRUE AND business_name != 'Restaurant' THEN restaurant_id ELSE NULL END) AS new_online_vertical,
        count(CASE WHEN was_online = TRUE AND confirmed_orders = 0 THEN restaurant_id ELSE NULL END) AS zombies,
        count(CASE WHEN was_online = TRUE AND confirmed_orders >= 200 THEN restaurant_id ELSE NULL END) AS high_orders_partners,
        count(CASE WHEN was_online = TRUE AND confirmed_orders <= 30 AND confirmed_orders > 0 THEN restaurant_id ELSE NULL END) AS low_orders_partners,
        count(CASE WHEN was_online = TRUE AND confirmed_orders < 200 AND confirmed_orders > 30 THEN restaurant_id ELSE NULL END) AS medium_orders_partners,
        
        
        CAST(sum(confirmed_orders) AS numeric)/CAST(sum(sessions) AS numeric) AS cvr_partners,
        if(registered_week = DATE_TRUNC(current_date(),isoweek),
        
        count(CASE WHEN was_online = TRUE AND is_online_present = FALSE THEN restaurant_id ELSE NULL END),
        
        NULL) AS possible_churn_next_week

        FROM
            
            
            (        
                
            -- selecciona de la tabla del WITH y genera la base
            
            
            SELECT 
            
            historical.city_id,
            historical.registered_week,
            historical.restaurant_id,
            last_day_history.is_online,
            last_day_history.is_logistic,
            last_day_history.business_name,
            if(count(DISTINCT CASE WHEN historical.is_online = TRUE THEN historical.restaurant_id ELSE NULL END) > 0,TRUE,FALSE) AS was_online,
            lag(if(count(DISTINCT CASE WHEN historical.is_online = TRUE THEN historical.restaurant_id ELSE NULL END) > 0,TRUE,FALSE)) 
            OVER (PARTITION BY historical.restaurant_id ORDER BY historical.registered_week ASC) AS online_last_week,
            if(count(DISTINCT CASE WHEN historical.is_new_online = TRUE THEN historical.restaurant_id ELSE NULL END) > 0,TRUE,FALSE) AS new_online,
            sum(sessions) AS sessions,
            if(confirmed_orders IS NOT NULL,confirmed_orders,0) AS confirmed_orders,
            partners.is_online AS is_online_present
            
            
            FROM
            
            historical
            
            -- sale del WITH para poder sacar el estado del último día
            
            LEFT JOIN (SELECT         
                    registered_week,
                    restaurant_id,
                    is_online,
                    is_logistic,
                    business_name
                    
                    FROM historical
                    
                    WHERE row_num = 1) last_day_history
                    
                    ON historical.restaurant_id = last_day_history.restaurant_id AND historical.registered_week = last_day_history.registered_week
            
            -- órdenes por partner
            
            LEFT JOIN (SELECT
                        DATE_TRUNC(registered_date, isoweek) AS registered_week,
                        restaurant. id AS restaurant_id,
                        count(order_id) AS confirmed_orders
                        
                        FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                        
                        
                        WHERE
                        registered_date >= DATE_SUB(DATE_TRUNC(current_date(), isoweek), INTERVAL 10 week)
                        AND country_id = 3
                        AND order_status = 'CONFIRMED'
                        
                        GROUP BY 
                        1,2
                        
                        ) orders
                        
                        ON historical.restaurant_id = orders.restaurant_id AND historical.registered_week = orders.registered_week
                        
            LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` partners ON historical.restaurant_id = partners.partner_id
          
            
                    
            GROUP BY 
            
            historical.city_id,
            historical.registered_week,
            historical.restaurant_id,
            last_day_history.is_online,
            last_day_history.is_logistic,
            last_day_history.business_name,
            confirmed_orders,
            partners.is_online
            
            
            
        )

        GROUP BY 1,2

        ORDER BY 1,2
    ) supply

    ON orders.city_id = supply.city_id AND orders.registered_week = supply.registered_week


    ORDER BY 1,3'''

#### QUERY MONTHLY

In [5]:
# 67GB
q_monthly = '''SELECT 

orders.city_id,
orders.city_name,
filler AS filler_yyyymm,
orders.month,
orders.total_orders,
orders.confirmed_orders,
orders.confirmed_orders_logistics,
orders.rejected_by_restaurant,
orders.rejected_by_peya,
orders.cancelled_by_user,
orders.users,
orders.acquisitions,
orders.activity_rate,
orders.restaurant_orders,
orders.vertical_orders,
orders.campaign_orders,
orders.confirmed_orders_voucher,
orders.vendor_late_numerator,
orders.vendor_late_denominator,
orders.avg_vendor_late,
orders.avg_delivery_time,
orders.dropoff_distance,
orders.pickup_distance,
orders.orders_less_20min,
orders.orders_late,
orders.stacked_orders,
nps.nps,
orders.gfv_inc_peya_discount_logistics,
orders.take_in_commission_logistics,
orders.commission_logistics,
orders.take_in_marketplace,
orders.commission_marketplace,
orders.shipping_amount,
orders.avg_basket_size,
demand.heavy_sh_users,
filler AS filler_filler,
demand.m1,
demand.m3,
demand.m6,
demand.no_return_users,
demand.returning_users,
supply.online_partners,
supply.od_partners,
supply.online_verticals,
supply.churned,
supply.new_online,
supply.zombies,
supply.high_orders_partners,
supply.cvr_partners,
nps.promoters,
nps.detractors,
nps.answers,
supply.new_online_logistic,
supply.new_online_vertical,
demand.returning_m1,
demand.acquisitions_m1,
demand.returning_m3,
demand.acquisitions_m3,
demand.returning_m6,
demand.acquisitions_m6,
orders.rejected_by_rider,
supply.possible_churn_next_month,
marketing_investment.cambio_usd,
marketing_investment.marketing_investment,
marketing_investment.marketing_investment_usd,
marketing_investment.subsidized_orders,
marketing_investment.investment_subsidies,
marketing_investment.cpo_campaigns,
user_base.user_base,
sessions.sessions,
sessions.sessions_shops,
sessions.sessions_w_transaction,
sessions.cvr3,
sessions.cvr,
sessions.direct_sessions,
sessions.crm_sessions,
sessions.cpc_sessions,
sessions.organic_sessions,
clusters.light_users,
clusters.medium_users,
clusters.heavy_users,
clusters.super_heavy_users,
clusters.light_users_orders,
clusters.medium_users_orders,
clusters.heavy_users_orders,
clusters.super_heavy_users_orders,
clusters.deficitary_users,
clusters.deficitary_users_orders,
clusters.spend_on_deficitary_users,
clusters.spend_on_users,
clusters.users_with_organic_orders,
demand.ltv_m5,
demand.ltv_revenue,
demand.ltv_commission_revenue,
demand.ltv_shipping_amount,
demand.ltv_cost,
demand.ltv_discount_paid_by_company,
demand.ltv_cpo,
demand.users_m5,
marketing_investment.investment_by_others_usd,
marketing_investment.subsidized_orders_by_others,
orders.total_revenue,
sessions.cvr_manana,
sessions.cvr_mediodia,
sessions.cvr_tarde,
sessions.cvr_noche,
commission_new_online.commission_new_food_logistic,
commission_new_online.commission_new_non_food_logistic,
commission_new_online.commission_new_food_marketplace,
commission_new_online.commission_new_non_food_marketplace,
demand.frequency_existing,
demand.frequency_returning,
demand.frequency_acquisitions,
supply.low_orders_partners,
supply.medium_orders_partners,
demand.acquisitions_rma,
demand.acquisitions_raf,
demand.acquisitions_organicos,
demand.m1_rma,
demand.m1_raf,
demand.m1_organicos,
last_data_date.last_data_date


FROM 

-- 1 ORDERS

        (SELECT 

            address.area.city. id AS city_id,
            cities.city_name,
            DATE_TRUNC(registered_date, month) AS month,

            -- órdenes generales

            count(orders.order_id) AS total_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS confirmed_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND with_logistics = TRUE THEN orders.order_id ELSE NULL END) AS confirmed_orders_logistics,
            count(CASE WHEN fail_rate_owner = 'Restaurant' THEN orders.order_id ELSE NULL END) AS rejected_by_restaurant,
            count(CASE WHEN fail_rate_owner = 'PedidosYa' THEN orders.order_id ELSE NULL END) AS rejected_by_peya,
            count(CASE WHEN fail_rate_owner = 'User' THEN orders.order_id ELSE NULL END) AS cancelled_by_user,
            count(CASE WHEN fail_rate_owner = 'Rider' THEN orders.order_id ELSE NULL END) AS rejected_by_rider, 

            -- demand y usuarios

            count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.user.id ELSE NULL END) AS users,
            count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' AND data.first = TRUE THEN orders.user.id ELSE NULL END) AS acquisitions,
            SAFE_DIVIDE(CAST(count(CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric),
            CAST(count(DISTINCT CASE WHEN orders.order_status = 'CONFIRMED' THEN orders.user.id ELSE NULL END) AS numeric)) AS activity_rate,


            -- verticales,vouchers,campañas
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND orders.business_type_id = 1 THEN orders.order_id ELSE NULL END) AS restaurant_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND orders.business_type_id != 1 THEN orders.order_id ELSE NULL END) AS vertical_orders,
            count(CASE WHEN subsidized_order IS NOT NULL THEN orders.order_id ELSE NULL END) AS campaign_orders,
            count(CASE WHEN orders.order_status = 'CONFIRMED' AND has_voucher_discount = 1 THEN orders.order_id ELSE NULL END) AS confirmed_orders_voucher,



            -- logística   

            count(CASE WHEN logs.is_vendor_late_10 = 1 THEN orders.order_id ELSE NULL END) AS vendor_late_numerator,
            count(CASE WHEN logs.is_vendor_late_nn = 1 THEN orders.order_id ELSE NULL END) AS vendor_late_denominator,
            avg(CASE WHEN logs.is_vendor_late_nn = 1 THEN SAFE_CAST(logs.timings.vendor_late AS numeric)/60 ELSE NULL END) AS avg_vendor_late,
            avg(CASE WHEN logs.timings.actual_delivery_time IS NOT NULL THEN SAFE_CAST(logs.timings.actual_delivery_time AS numeric)/60 ELSE NULL END) AS avg_delivery_time,
            avg(CASE WHEN orders.order_status = 'CONFIRMED' AND dropoff_distance_manhattan IS NOT NULL THEN dropoff_distance_manhattan ELSE NULL END) AS dropoff_distance,
            avg(CASE WHEN orders.order_status = 'CONFIRMED' AND pickup_distance_manhattan IS NOT NULL THEN pickup_distance_manhattan ELSE NULL END) AS pickup_distance,
            count(CASE WHEN SAFE_CAST(logs.timings.actual_delivery_time AS numeric)/60 <= 20 AND logs.timings.actual_delivery_time IS NOT NULL AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS orders_less_20min,
            count(CASE WHEN logs.is_order_late_10 = 1 AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS orders_late,
            count(CASE WHEN logistic_deliveries.stacked_deliveries > 0 AND logistic_deliveries.stacked_deliveries IS NOT NULL AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS stacked_orders,


            -- economics

            sum(CASE WHEN with_logistics = TRUE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                        / CAST(count(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS gfv_inc_peya_discount_logistics,
                        
            sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) 
                        / CAST(count(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS take_in_commission_logistics,
                        
            sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) /
            sum(CASE WHEN with_logistics = TRUE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                        AS commission_logistics,
                        
            sum(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) 
                        / CAST(count(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN orders.order_id ELSE NULL END) AS numeric) AS take_in_marketplace,
                        
            sum(CASE WHEN with_logistics = FALSE AND orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END) /
            sum(CASE WHEN with_logistics = FALSE AND commission_amount IS NOT NULL AND orders.commission IS NOT NULL AND orders.commission != 0 AND orders.order_status = 'CONFIRMED' THEN commission_amount / (orders.commission/100) ELSE NULL END)
                        AS commission_marketplace,
                        
            avg(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN shipping_amount ELSE NULL END) AS shipping_amount,
                        
            avg(CASE WHEN orders.order_status = 'CONFIRMED' THEN total_amount+shipping_amount ELSE NULL END) AS avg_basket_size,
                        
            sum(CASE WHEN orders.order_status = 'CONFIRMED' THEN commission_amount ELSE NULL END)+sum(CASE WHEN with_logistics = TRUE AND orders.order_status = 'CONFIRMED' THEN shipping_amount ELSE NULL END) AS total_revenue
            



            FROM `peya-bi-tools-pro.il_core.fact_orders` orders

                LEFT JOIN `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS logs
                
                    ON orders.order_id = logs.peya_order_id
                        
                LEFT JOIN (
                        
                            SELECT peya_order_id, sum(stacked_deliveries) AS stacked_deliveries, avg(pickup_distance_manhattan) AS pickup_distance_manhattan,avg(dropoff_distance_manhattan) AS dropoff_distance_manhattan
                            
                            FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` logs

                            LEFT JOIN UNNEST(logs.deliveries) AS details
                            
                            GROUP BY peya_order_id
                            
                            ) logistic_deliveries

                    ON orders.order_id = logistic_deliveries.peya_order_id

                
                
                -- CAMPAIGN ORDERS

                    LEFT JOIN (SELECT DISTINCT order_id AS subsidized_order

                                    FROM `peya-bi-tools-pro.il_core.fact_orders` o

                                    LEFT JOIN UNNEST(o.details) AS details


                                    JOIN peya-bi-tools-pro.il_core.dim_subsidized_product AS dim_subProd     
                                    ON details.product.product_id = dim_subProd.product_id

                                    WHERE o.registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
                                    AND o.country_id = 3) campaigns
                    
                    ON orders.order_id = campaigns.subsidized_order


                    -- CITIES

                    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` cities

                    ON orders.address.area.city.id = cities.city_id

            WHERE 

            registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
            AND orders.country_id = 3

            GROUP BY 
            1,2,3
            
            ) orders


-- 2 DEMAND

LEFT JOIN (    
    
        -- hace los cálculos

                SELECT
                
                city_id,
                month,
                
                count(CASE WHEN orders > 0 AND orders IS NOT NULL THEN user ELSE NULL END) AS users_tot,
                count(CASE WHEN orders >= 5 AND orders IS NOT NULL THEN user ELSE NULL END) AS heavy_sh_users,

                SAFE_DIVIDE(CAST(count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance  = 'acquisition' THEN user ELSE NULL END) AS numeric),
                CAST(count(CASE WHEN lag_existance  = 'acquisition' THEN user ELSE NULL END) AS numeric)) AS m1,

                SAFE_DIVIDE(CAST(count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance3  = 'acquisition' THEN user ELSE NULL END) AS numeric),
                CAST(count(CASE WHEN lag_existance3  = 'acquisition' THEN user ELSE NULL END) AS numeric)) AS m3,

                SAFE_DIVIDE(CAST(count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance6  = 'acquisition' THEN user ELSE NULL END) AS numeric),
                CAST(count(CASE WHEN lag_existance6  = 'acquisition' THEN user ELSE NULL END) AS numeric)) AS m6,
                
                count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance  = 'acquisition' THEN user ELSE NULL END) AS returning_m1,
                count(CASE WHEN lag_existance  = 'acquisition' THEN user ELSE NULL END)   AS acquisitions_m1,
                count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance3  = 'acquisition' THEN user ELSE NULL END) AS returning_m3,
                count(CASE WHEN lag_existance3  = 'acquisition' THEN user ELSE NULL END)   AS acquisitions_m3,
                count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance6  = 'acquisition' THEN user ELSE NULL END) AS returning_m6,
                count(CASE WHEN lag_existance6  = 'acquisition' THEN user ELSE NULL END)   AS acquisitions_m6,
                
                count(CASE WHEN existance_status = 'existing' AND (lag_existance  IN ('existing','acquisition'))
                AND (lag_orders > 0 AND lag_orders IS NOT NULL) AND (orders IS NULL OR orders = 0) THEN user ELSE NULL END) AS no_return_users,
                
                count(CASE WHEN existance_status = 'existing' AND (lag_existance  = 'existing')
                AND (orders > 0 AND orders IS NOT NULL) AND (lag_orders IS NULL OR lag_orders = 0) THEN user ELSE NULL END) AS returning_users,
                
                avg(CASE WHEN mx = 5 THEN running_ltv ELSE NULL END) AS ltv_m5,
                avg(CASE WHEN mx = 5 THEN r_revenue ELSE NULL END) AS ltv_revenue,
                avg(CASE WHEN mx = 5 THEN r_commission_revenue ELSE NULL END) AS ltv_commission_revenue,
                avg(CASE WHEN mx = 5 THEN r_revenue ELSE NULL END) - avg(CASE WHEN mx = 5 THEN r_commission_revenue ELSE NULL END) AS ltv_shipping_amount,
                avg(CASE WHEN mx = 5 THEN r_cost ELSE NULL END) AS ltv_cost,
                avg(CASE WHEN mx = 5 THEN r_discount_paid_by_company ELSE NULL END) AS ltv_discount_paid_by_company,
                avg(CASE WHEN mx = 5 THEN r_cost ELSE NULL END) - avg(CASE WHEN mx = 5 THEN r_discount_paid_by_company ELSE NULL END) AS ltv_cpo,
                count(CASE WHEN mx = 5 THEN user ELSE NULL END) AS users_m5,    
                
                avg(CASE WHEN existance_status = 'existing' AND orders > 0 AND orders IS NOT NULL THEN orders ELSE NULL END) AS frequency_existing,
                avg(CASE WHEN existance_status = 'existing' AND (lag_existance  = 'existing')
                AND (orders > 0 AND orders IS NOT NULL) AND (lag_orders IS NULL OR lag_orders = 0) THEN orders ELSE NULL END) AS frequency_returning,
                avg(CASE WHEN existance_status = 'acquisition' AND orders > 0 AND orders IS NOT NULL THEN orders ELSE NULL END) AS frequency_acquisitions,
                
                count(CASE WHEN existance_status = 'acquisition' AND c_type = 'RMA' THEN orders ELSE NULL END) AS acquisitions_rma,
                count(CASE WHEN existance_status = 'acquisition' AND c_type = 'RAF' THEN orders ELSE NULL END) AS acquisitions_raf,
                count(CASE WHEN existance_status = 'acquisition' AND c_type = 'Organicos' THEN orders ELSE NULL END) AS acquisitions_organicos,
                
                SAFE_DIVIDE(CAST(count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance  = 'acquisition' AND c_type = 'RMA' THEN user ELSE NULL END) AS numeric),
                CAST(count(CASE WHEN lag_existance  = 'acquisition' AND c_type = 'RMA'  THEN user ELSE NULL END) AS numeric)) AS m1_rma,
                SAFE_DIVIDE(CAST(count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance  = 'acquisition' AND c_type = 'RAF' THEN user ELSE NULL END) AS numeric),
                CAST(count(CASE WHEN lag_existance  = 'acquisition' AND c_type = 'RAF'  THEN user ELSE NULL END) AS numeric)) AS m1_raf,
                SAFE_DIVIDE(CAST(count(CASE WHEN orders > 0 AND orders IS NOT NULL AND lag_existance  = 'acquisition' AND c_type = 'Organicos' THEN user ELSE NULL END) AS numeric),
                CAST(count(CASE WHEN lag_existance  = 'acquisition' AND c_type = 'Organicos'  THEN user ELSE NULL END) AS numeric)) AS m1_organicos
                
                
                FROM
                
                (
                    
                    -- junta toda la información del usuario por mes, sus órdenes y si estaba vivo pero le suma sus órdenes y existencia del mes anterior

                    SELECT
                            
                            city_id,
                            month,
                            existance_status,
                            first_month,
                            c_type,
                            DATE_DIFF(month,first_month,MONTH) AS mx,
                            user,
                            orders,
                            lag(existance_status) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS lag_existance,
                            lag(orders) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS lag_orders,
                            lag(existance_status,3) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS lag_existance3,
                            lag(existance_status,6) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS lag_existance6,
                            profit,
                            sum(profit) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS running_ltv,
                            sum(revenue) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS r_revenue,
                            sum(cost) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS r_cost,
                            sum(discount_paid_by_company) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS r_discount_paid_by_company,
                            sum(cpo) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS r_cpo,
                            sum(commission_revenue) OVER(PARTITION BY user,city_id ORDER BY month ASC) AS r_commission_revenue
                            
                            FROM
                            
                            -- junta toda la información del usuario por mes, sus órdenes y si estaba vivo
                                    
                                    (SELECT 
                                    users.user,
                                    users.city_id,
                                    users.month,
                                    first_month,
                                    c_type,
                                    
                                    if(users.month = first_month AND users.city_id = first_city,'acquisition',
                                    if(users.month = first_month AND users.city_id != first_city,'existing',
                                    if(users.month < first_month,'unborn',
                                    if(users.month > first_month,'existing', 
                                    if(first_month IS NULL,'existing',NULL))))) AS existance_status,
                                    
                                    confirmed_orders AS orders,
                                    profit,
                                    revenue,
                                    cost,
                                    discount_paid_by_company,
                                    cpo,
                                    commission_revenue

                                    FROM

                                    -- selecciona todos los usuarios que compraron y les asigna todos los meses

                                            (SELECT 
                                            
                                            month,user,city_id
                                            
                                            FROM
                                            
                                            (SELECT DISTINCT
                                            
                                            date_trunc(registered_date, month) AS month
                                            
                                            FROM `peya-bi-tools-pro.il_core.fact_orders` orders 
                                            
                                            WHERE
                                            
                                            registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 11 month)
                                            )
                                            
                                            
                                            CROSS JOIN 
                                            
                                            (SELECT DISTINCT orders.user.id AS user, address.area.city. id AS city_id
                                            FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                                            WHERE
                                            registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 11 month)
                                            AND country_id = 3
                                            AND order_status = 'CONFIRMED'
                                            )
                                            
                                            ) users
                                    -- termina la asignación de usuarios

                                    -- busca el mes de adquisición del usuario, la ciudad donde compró primero y el tipo de adquisición
                                    
                                    LEFT JOIN 
                                    
                                        (SELECT DISTINCT 
                                        orders.user.id AS user_first, 
                                        date_trunc(registered_date, month) AS first_month, 
                                        address.area.city. id AS first_city,
                                        coupon_used_amount,
                                        CASE WHEN talon_campaign_name LIKE '%%RMA%%' THEN 'RMA'
                                            WHEN talon_campaign_name LIKE '%%RAF%%' THEN 'RAF'
                                            WHEN talon_campaign_name IS NULL THEN 'Organicos'
                                            WHEN talon_campaign_name LIKE '%%RMO%%' THEN 'RMA'
                                            ELSE 'RMA' END as c_type
                                        
                                        FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                                                    
                                        LEFT JOIN
                                    
                                                                (
                                                                SELECT
                                                                order_id,
                                                                talon_campaign_id,
                                                                coupon_used_amount
                                                            
                                                                FROM `peya-bi-tools-pro.il_growth.fact_talon_coupons`
                                                            
                                                                WHERE
                                                                DATE(order_registered_date) >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 11 month)
                                                                AND coupon_country_id = 3
                                                            
                                                                ) coupons
                                
                                                        ON orders.order_id = coupons.order_id
                                                        
                                                        LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` talon_campaigns ON talon_campaigns.talon_campaign_id = coupons.talon_campaign_id
                                                        
                                                    WHERE
                                                    registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 11 month)
                                                    AND country_id = 3
                                                    AND data.first = TRUE 
                                                    AND order_status = 'CONFIRMED'
                                                    ) first
                                        
                                        
                                        ON user = user_first

                                -- saca las órdenes por usuario
                                    
                                    LEFT JOIN 
                                    
                                        (SELECT 
                                        
                                        
                                        orders.user.id AS user, 
                                        date_trunc(registered_date, month) AS month, 
                                        address.area.city.id AS city_id,

                                        count(order_id) AS confirmed_orders,

                                        -- NO ESTA EL DATO CPO SACADO!!! como no está el CPO el placeholder es cpo = commission_amount*0.8
                                        sum(CASE WHEN with_logistics = TRUE THEN (commission_amount + shipping_amount) - (discount_paid_by_company + commission_amount*0.8) ELSE commission_amount - discount_paid_by_company END)/usd AS profit,
                                        sum(CASE WHEN with_logistics = TRUE THEN (commission_amount) ELSE commission_amount END)/usd AS commission_revenue,
                                        sum(CASE WHEN with_logistics = TRUE THEN (commission_amount + shipping_amount) ELSE commission_amount END)/usd AS revenue,
                                        sum(CASE WHEN with_logistics = TRUE THEN (discount_paid_by_company + commission_amount*0.8) ELSE discount_paid_by_company END)/usd AS cost,
                                        sum(CASE WHEN with_logistics = TRUE THEN discount_paid_by_company ELSE discount_paid_by_company END)/usd discount_paid_by_company,
                                        sum(CASE WHEN with_logistics = TRUE THEN commission_amount*0.8 ELSE NULL END)/usd AS cpo
                                        
                                        
                                        
                                        FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                                            
                                            -- trae el cpo NO ESTA EL DATO CPO SACADO!!!

                                            -- para sacar el valor del dolaruco
                                            LEFT JOIN (SELECT currency_exchange_date, rate_us AS usd FROM `peya-bi-tools-pro.il_core.dim_currency_exchange`

                                            WHERE currency_id = 3) exchange
                                            
                                            ON date_trunc(registered_date, month) = exchange.currency_exchange_date
                                                            
                                        WHERE   
                                        
                                        registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 11 month)
                                        AND country_id = 3
                                        AND order_status = 'CONFIRMED'
                                        
                                        GROUP BY 
                                        orders.user.id,
                                        date_trunc(registered_date, month), 
                                        address.area.city.id,
                                        usd
                                        ) orders
                                    
                                    ON users.month = orders.month AND users.user = orders.user AND users.city_id = orders.city_id

                                )
                            
                            )

                GROUP BY 
                city_id,
                month

                ORDER BY 1,2
                
                ) demand

                ON orders.city_id = demand.city_id AND orders.month = demand.month


-- 3 NPS

LEFT JOIN  (
               SELECT
            
            address.area.city. id AS city_id,
            DATE_TRUNC(registered_date, month) AS month,
            
            -- nps
            avg(if(nps_score>=9,100,if(nps_score>=7,0,if(nps_score>=0 AND nps_score IS NOT NULL,-100,NULL)))) AS nps,
            count(CASE WHEN nps_score >= 9 AND nps_score IS NOT NULL THEN orders.order_id ELSE NULL END) AS promoters,
            count(CASE WHEN nps_score <= 6 AND nps_score IS NOT NULL THEN orders.order_id ELSE NULL END) AS detractors,
            count(CASE WHEN nps_score IS NOT NULL THEN orders.order_id ELSE NULL END) AS answers
            
            FROM `peya-bi-tools-pro.il_core.fact_orders` orders
            
            -- nps
            
            LEFT JOIN (SELECT DISTINCT order_id, nps_score FROM `peya-bi-tools-pro.il_nps.fact_nps_ao_related_kpis`) nps_ao_related_kpis
            
            ON orders.order_id = nps_ao_related_kpis.order_id
            
            WHERE 
            
            registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
            AND country_id = 3
            
            
            GROUP BY
            
            1,2

            ORDER BY 1,2
             
        ) nps

        ON orders.city_id = nps.city_id AND orders.month = nps.month


-- 4 SUPPLY

    LEFT JOIN (

            -- saca los datos de status por día para ver si estuvo online y algunas cosillas mas
                    
            WITH historical AS (

                SELECT 
                    
                    partners_historical.city_id,
                    DATE_TRUNC(DATE(full_date),month) AS month,
                    DATE(partners_historical.full_date) AS date,
                    partners_historical.restaurant_id,
                    partners_historical.is_online,
                    is_logistic,
                    is_new_online,
                    business_name,
                    sessions,
                    row_number() OVER (PARTITION BY DATE_TRUNC(DATE(full_date),month),partners_historical.restaurant_id ORDER BY DATE(partners_historical.full_date) DESC) AS row_num
                    
                    FROM `peya-bi-tools-pro.il_core.dim_historical_partners` partners_historical
                    
                    
                    -- sesiones
                    
                    LEFT JOIN `peya-bi-tools-pro.il_sessions.fact_sessions_by_restaurant` sessions_by_restaurant
                    
                    ON partners_historical.restaurant_id = sessions_by_restaurant.restaurant_id
                    AND DATE(partners_historical.full_date) = sessions_by_restaurant.date
                    
                    WHERE 
                        DATE(partners_historical.full_date) >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
                        AND partners_historical.country_id = 3

            )   


    SELECT
    
        city_id,
        month,
        count(CASE WHEN was_online = TRUE THEN restaurant_id ELSE NULL END) AS online_partners,
        count(CASE WHEN was_online = TRUE AND is_logistic = TRUE THEN restaurant_id ELSE NULL END) AS od_partners,
        count(CASE WHEN was_online = TRUE AND business_name != 'Restaurant' THEN restaurant_id ELSE NULL END) AS online_verticals,
        count(CASE WHEN was_online = FALSE AND online_last_month = TRUE THEN restaurant_id ELSE NULL END) AS churned,
        count(CASE WHEN new_online = TRUE THEN restaurant_id ELSE NULL END) AS new_online,
        count(CASE WHEN new_online = TRUE AND is_logistic = TRUE THEN restaurant_id ELSE NULL END) AS new_online_logistic,
        count(CASE WHEN new_online = TRUE AND business_name != 'Restaurant' THEN restaurant_id ELSE NULL END) AS new_online_vertical,
        count(CASE WHEN was_online = TRUE AND confirmed_orders = 0 THEN restaurant_id ELSE NULL END) AS zombies,
        count(CASE WHEN was_online = TRUE AND confirmed_orders >= 200 THEN restaurant_id ELSE NULL END) AS high_orders_partners,
        count(CASE WHEN was_online = TRUE AND confirmed_orders <= 30 AND confirmed_orders > 0 THEN restaurant_id ELSE NULL END) AS low_orders_partners,
        count(CASE WHEN was_online = TRUE AND confirmed_orders < 200 AND confirmed_orders > 30 THEN restaurant_id ELSE NULL END) AS medium_orders_partners,
        
        
        CAST(sum(confirmed_orders) AS numeric)/CAST(sum(sessions) AS numeric) AS cvr_partners,
        if(month = DATE_TRUNC(current_date(),month),
        
        count(CASE WHEN was_online = TRUE AND is_online_present = FALSE THEN restaurant_id ELSE NULL END),
        
        NULL) AS possible_churn_next_month

        FROM
            
            
            (        
                
            -- selecciona de la tabla del WITH y genera la base
            
            
            SELECT 
            
            historical.city_id,
            historical.month,
            historical.restaurant_id,
            last_day_history.is_online,
            last_day_history.is_logistic,
            last_day_history.business_name,
            if(count(DISTINCT CASE WHEN historical.is_online = TRUE THEN historical.restaurant_id ELSE NULL END) > 0,TRUE,FALSE) AS was_online,
            lag(if(count(DISTINCT CASE WHEN historical.is_online = TRUE THEN historical.restaurant_id ELSE NULL END) > 0,TRUE,FALSE)) 
            OVER (PARTITION BY historical.restaurant_id ORDER BY historical.month ASC) AS online_last_month,
            if(count(DISTINCT CASE WHEN historical.is_new_online = TRUE THEN historical.restaurant_id ELSE NULL END) > 0,TRUE,FALSE) AS new_online,
            sum(sessions) AS sessions,
            if(confirmed_orders IS NOT NULL,confirmed_orders,0) AS confirmed_orders,
            partners.is_online AS is_online_present
            
            
            FROM
            
            historical
            
            -- sale del WITH para poder sacar el estado del último día
            
            LEFT JOIN (SELECT         
                    month,
                    restaurant_id,
                    is_online,
                    is_logistic,
                    business_name
                    
                    FROM historical
                    
                    WHERE row_num = 1) last_day_history
                    
                    ON historical.restaurant_id = last_day_history.restaurant_id AND historical.month = last_day_history.month
            
            -- órdenes por partner
            
            LEFT JOIN (SELECT
                        DATE_TRUNC(registered_date, month) AS month,
                        restaurant. id AS restaurant_id,
                        count(order_id) AS confirmed_orders
                        
                        FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                        
                        
                        WHERE
                        registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
                        AND country_id = 3
                        AND order_status = 'CONFIRMED'
                        
                        GROUP BY 
                        1,2
                        
                        ) orders
                        
                        ON historical.restaurant_id = orders.restaurant_id AND historical.month = orders.month
                        
            LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` partners ON historical.restaurant_id = partners.partner_id
          
            
                    
            GROUP BY 
            
            historical.city_id,
            historical.month,
            historical.restaurant_id,
            last_day_history.is_online,
            last_day_history.is_logistic,
            last_day_history.business_name,
            confirmed_orders,
            partners.is_online
            
            
            
        )

        GROUP BY 1,2

        ORDER BY 1,2
    ) supply


    ON orders.city_id = supply.city_id AND orders.month = supply.month 


-- 5 MARKETING INVESTMENT

LEFT JOIN (

            SELECT 
    
                address.area.city. id AS city_id,
                DATE_TRUNC(registered_date, month) AS month,
                usd AS cambio_usd,
                
                sum(CASE WHEN discounts.discount_paid_by = 'COMPANY' AND discount_type_name IN ('VOUCHER','SUBSIDIZED') THEN discounts. discount_amount ELSE NULL END) AS marketing_investment, 
                SAFE_DIVIDE(sum(CASE WHEN discounts.discount_paid_by = 'COMPANY' AND discount_type_name IN ('VOUCHER','SUBSIDIZED') THEN discounts. discount_amount ELSE NULL END),usd) AS marketing_investment_usd, 
                count(CASE WHEN discounts.discount_paid_by = 'COMPANY' AND discount_type_name IN ('VOUCHER','SUBSIDIZED') THEN discounts. order_id	 ELSE NULL END) AS subsidized_orders,
                SAFE_DIVIDE(sum(CASE WHEN discount_type_name = 'SUBSIDIZED' AND discounts.discount_paid_by = 'COMPANY' THEN discounts. discount_amount ELSE NULL END),usd) AS investment_subsidies,
                
                SAFE_DIVIDE(SAFE_DIVIDE(sum(CASE WHEN discount_type_name = 'SUBSIDIZED'AND  discounts.discount_paid_by = 'COMPANY' THEN discounts. discount_amount ELSE NULL END),usd),
                COUNT(CASE WHEN discount_type_name = 'SUBSIDIZED' AND discounts.discount_paid_by = 'COMPANY' THEN discounts. order_id	 ELSE NULL END)) AS cpo_campaigns,
                
                SAFE_DIVIDE(sum(CASE WHEN discounts.discount_paid_by = 'RESTAURANT' THEN discounts. discount_amount ELSE NULL END),usd) AS investment_by_others_usd,
                COUNT(CASE WHEN discounts.discount_paid_by = 'RESTAURANT' THEN discounts. order_id	 ELSE NULL END) AS subsidized_orders_by_others
                
                
                FROM `peya-bi-tools-pro.il_core.fact_orders` o

                LEFT JOIN UNNEST (o.discounts) discounts

                -- para sacar el valor del dolaruco
                LEFT JOIN (SELECT currency_exchange_date, rate_us AS usd FROM `peya-bi-tools-pro.il_core.dim_currency_exchange`

                WHERE currency_id = 3) exchange
                                                
                ON date_trunc(registered_date, month) = exchange.currency_exchange_date


                WHERE
                registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
                AND country_id = 3

                GROUP BY 1,2,3

                ORDER BY 1,2
        ) marketing_investment


        ON orders.city_id = marketing_investment.city_id AND orders.month = marketing_investment.month


-- 6 ACTIVITY RATE Y USER BASE

LEFT JOIN (

            SELECT 
orders.city_id,
orders.month,
sum(IF(orders.month='2021-01-01',0,orders.acquisitions) + COALESCE(user_base.user_base,0)) OVER (PARTITION BY orders.city_id ORDER BY orders.month) AS user_base

FROM
-- le va sacando las adquisiciones a partir desde enero 2021, todas las anteriores salen de la tabla reporting-regiones-310112.user_base.user_base_arg_20210101 que me cree antes con datos del reporte de ofi 1
        
            (SELECT 
                
                    address.area.city. id AS city_id,
                    DATE_TRUNC(registered_date, month) AS month,
                    count(orders.user.id) AS acquisitions
                    
                    FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                    
                    WHERE
                    
                    
                    registered_date >= '2021-01-01'
                    AND country_id = 3
                    AND data.first = TRUE
                    AND order_status = 'CONFIRMED'
                    
                    GROUP BY 1,2
                    ) orders
                    
                    
                    LEFT JOIN `peya-argentina.user_santiago_curat.user_base_arg_20210101` user_base

                    ON orders.city_id = user_base.city_id AND orders.month = user_base.month



            ORDER BY 1,2

        ) user_base

        ON orders.city_id = user_base.city_id AND orders.month = user_base.month



-- 7 SESIONES Y CVR

LEFT JOIN (

                SELECT 
            
            cdcity,
            DATE_TRUNC(date_parsed,MONTH) AS month,
            count(visitid) AS sessions,
            
            -- cvrs
            
            count(CASE WHEN uniqueshop >= 1 THEN visitid ELSE NULL END) AS sessions_shops,
            count(CASE WHEN transaction >= 1 THEN visitid ELSE NULL END) AS sessions_w_transaction,
            SAFE_DIVIDE(count(CASE WHEN transaction >= 1 THEN visitid ELSE NULL END),CAST(count(CASE WHEN uniqueshop >= 1 THEN visitid ELSE NULL END) AS numeric)) AS cvr3,
            SAFE_DIVIDE(count(CASE WHEN transaction >= 1 THEN visitid ELSE NULL END),CAST(count(visitid) AS numeric)) AS cvr,
            
            -- session source
            count(CASE WHEN source = '(direct)' THEN visitid ELSE NULL END) AS direct_sessions,
            count(CASE WHEN lower(medium) = 'crm' THEN visitid ELSE NULL END) AS crm_sessions,
            count(CASE WHEN medium = 'cpc' THEN visitid ELSE NULL END) AS cpc_sessions,
            count(CASE WHEN medium = 'organic' THEN visitid ELSE NULL END) AS organic_sessions,
            
            -- cvrs hour
            
            SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=0 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<11  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=0 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<11  THEN visitid ELSE null END) AS numeric)) AS cvr_manana,
            SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=11 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<15  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=11 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<15  THEN visitid ELSE null END) AS numeric)) AS cvr_mediodia,
            SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=15 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<19  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=15 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<19  THEN visitid ELSE null END) AS numeric)) AS cvr_tarde,
            SAFE_DIVIDE(COUNT(CASE WHEN transaction >= 1 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=19 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<24  THEN visitid ELSE null END),CAST(COUNT(CASE WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))>=19 AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(visitStartTime),'America/Buenos_Aires'))<24  THEN visitid ELSE null END) AS numeric)) AS cvr_noche
            
            
            FROM `peya-bi-tools-pro.il_sessions.fact_sessions`
            
            
            WHERE 
            
            date_parsed >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
            AND cdCountry = 'Argentina'
            
            GROUP BY
            
            1,2

    ) sessions

    ON orders.city_name = sessions.cdcity AND orders.month = sessions.month 

-- 8 DEFICITARY USERS N' USER CLUSTERS

LEFT JOIN (

        SELECT
            
            city_id,
            month,
            count(CASE WHEN cluster = 'Light' THEN user_id ELSE NULL END) AS light_users,
            count(CASE WHEN cluster = 'Medium' THEN user_id ELSE NULL END) AS medium_users,
            count(CASE WHEN cluster = 'Heavy' THEN user_id ELSE NULL END) AS heavy_users,
            count(CASE WHEN cluster = 'Super Heavy' THEN user_id ELSE NULL END) AS super_heavy_users,
            sum(CASE WHEN cluster = 'Light' THEN confirmed_orders ELSE NULL END) AS light_users_orders,
            sum(CASE WHEN cluster = 'Medium' THEN confirmed_orders ELSE NULL END) AS medium_users_orders,
            sum(CASE WHEN cluster = 'Heavy' THEN confirmed_orders ELSE NULL END) AS heavy_users_orders,
            sum(CASE WHEN cluster = 'Super Heavy' THEN confirmed_orders ELSE NULL END) AS super_heavy_users_orders,
            
            count(CASE WHEN profit < 0 THEN user_id ELSE NULL END) AS deficitary_users,
            sum(CASE WHEN profit < 0 THEN confirmed_orders ELSE NULL END) AS deficitary_users_orders,
            sum(CASE WHEN profit < 0 THEN marketing_spend ELSE NULL END) AS spend_on_deficitary_users,
            sum(marketing_spend) AS spend_on_users,
            count(CASE WHEN confirmed_organic > 0 THEN user_id ELSE NULL END) AS users_with_organic_orders
            
            
            FROM
                
                
                
                (SELECT
                    
                    orders.user.id AS user_id,
                    address.area.city. id AS city_id,
                    DATE_TRUNC(registered_date, month) AS month,
                    count(order_id) AS confirmed_orders,
                    
                    IF(count(order_id)>=10,'Super Heavy',
                    
                    IF(count(order_id)>=5,'Heavy',
                    
                    IF(count(order_id)>=3,'Medium',
                    
                    IF(count(order_id)>=1,'Light',NULL)))) AS cluster,
                    
                    
                    sum(CASE WHEN with_logistics = TRUE THEN (commission_amount + shipping_amount) - (discount_paid_by_company + commission_amount*0.8) ELSE commission_amount - discount_paid_by_company END)/usd AS profit,
                    
                    sum(discount_paid_by_company)/usd AS marketing_spend,
                    
                    count(CASE WHEN has_voucher_discount = 0 THEN orders.order_id ELSE NULL END) AS confirmed_organic
                    
                    FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                    
                    
                    
                    -- ACÁ UNIR CON CPO .. FALTA CPO!!! (placeholde commission_amount*0.8)
                    
                    
                    -- para sacar el valor del dolaruco

                    LEFT JOIN (SELECT currency_exchange_date, rate_us AS usd FROM `peya-bi-tools-pro.il_core.dim_currency_exchange`

                    WHERE currency_id = 3) exchange
                                                    
                    ON date_trunc(registered_date, month) = exchange.currency_exchange_date
                        
                    
                    WHERE
                    
                    registered_date >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
                    AND country_id = 3
                    AND order_status = 'CONFIRMED'
                    
                    
                    GROUP BY 
                    
                    1,2,3,
                    usd
                    )

            GROUP BY 1,2
        
        ) clusters

        ON orders.city_id = clusters.city_id AND orders.month = clusters.month

-- 9 COMMISSION NEW ONLINE

LEFT JOIN (

       SELECT 

        /* comisión por ciudad, mes, grupo y log/mp*/

        city_id,
        month,
        avg(CASE WHEN business_group = 'Food' AND is_logistic = TRUE THEN commission ELSE NULL END) AS commission_new_food_logistic,
        avg(CASE WHEN business_group = 'Non Food' AND is_logistic = TRUE THEN commission ELSE NULL END) AS commission_new_non_food_logistic,
        avg(CASE WHEN business_group = 'Food' AND is_logistic = FALSE THEN commission ELSE NULL END) AS commission_new_food_marketplace,
        avg(CASE WHEN business_group = 'Non Food' AND is_logistic = FALSE THEN commission ELSE NULL END) AS commission_new_non_food_marketplace


        FROM


                (SELECT 
                        p.salesforce_id AS Grid,
                        DATE_TRUNC(DATE(cw.closed_date),month) AS month,
                        p.is_logistic,
                        p.city_id,
                        IF(p.business_type.business_type_name IN ('Restaurant','Coffee'),'Food','Non Food') AS business_group,
                        MAX(sfc.Commission) AS commission
                FROM peya-bi-tools-pro.il_core.fact_sales_executives_closed_won_v2 AS cw
                LEFT JOIN peya-bi-tools-pro.il_core.dim_partner AS p ON cw.grid = p.salesforce_id
                LEFT JOIN peya-data-origins-pro.cl_salesforce.opportunity AS sfo ON cw.oportunity_unique_id = sfo.Id
                LEFT JOIN peya-data-origins-pro.cl_salesforce.contract AS sfc ON sfo.Id = sfc.Id_Opportunity
                WHERE DATE(cw.closed_date) >= DATE_SUB(DATE_TRUNC(current_date(), month), INTERVAL 7 month)
                    AND p.country_id = 3
                    AND sfo.Business_Type IN ('New Business','Win Back','Franchise Extension','New Bussiness')
                GROUP BY 1,2,3,4,5)

            GROUP BY 1,2
        ) commission_new_online

        ON orders.city_id = commission_new_online.city_id AND orders.month = commission_new_online.month


-- 10 LAST DATA DATE

LEFT JOIN (

            SELECT DATE_TRUNC(registered_date, month) AS month, max(registered_date) AS last_data_date, max(registered_date) AS filler
                FROM `peya-bi-tools-pro.il_core.fact_orders` orders
                
                WHERE registered_date >= DATE_TRUNC(current_date(),month)
                
                GROUP BY 1

    ) last_data_date


    ON orders.month = last_data_date.month


ORDER BY 1,4'''

In [6]:
# Descargo la data
hue_30 = pd.io.gbq.read_gbq(q_30, project_id='peya-argentina', dialect='standard')

Downloading: 100%|████████████████████████████████████████████████████████████████| 297/297 [00:01<00:00, 281.29rows/s]


In [7]:
hue_weekly = pd.io.gbq.read_gbq(q_weekly, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████████| 2719/2719 [00:03<00:00, 804.58rows/s]


In [8]:
hue_monthly = pd.io.gbq.read_gbq(q_monthly, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████████| 2319/2319 [00:07<00:00, 323.71rows/s]


In [9]:
# Copio las bases
days_30 = hue_30.copy()
weekly = hue_weekly.copy()
monthly = hue_monthly.copy()

### TRABAJO

In [10]:
# Cambio formato a las columnas
monthly[['month','filler_yyyymm']] = monthly[['month','filler_yyyymm']].astype(str)
weekly['registered_week'] = weekly['registered_week'].astype(str)

### CARGA

In [11]:
# Carga Monthly
sheet_id = '1LSE_bBwgL8DIOL_RWLun-sRe0snaF2mctYHPEBJbQnc'
wks_name = 'Monthly'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(monthly, index=False, sheet=wks_name, replace=True)

In [12]:
# Carga Weekly
sheet_id = '1LSE_bBwgL8DIOL_RWLun-sRe0snaF2mctYHPEBJbQnc'
wks_name = 'Weekly'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(weekly, index=False, sheet=wks_name, replace=True)

In [13]:
# Carga 30 Days
sheet_id = '1LSE_bBwgL8DIOL_RWLun-sRe0snaF2mctYHPEBJbQnc'
wks_name = '30_Days'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(days_30, index=False, sheet=wks_name, replace=True)